# Código y trabajo

Parte 2
-----


**Restricción:** `No usar librerías especializadas, excepto numpy`

In [6]:
import numpy as np
import random as rd

### Enunciado
* Escriba un programa que permita entrenar una red FF con 1 capa escondida (H neuronas) y O neuronas de salida, **sin usar librerías**, excepto eventualmente *numpy* para implementar operaciones básicas de álgebra lineal. Por simplicidad, asuma que todas las neuronas implementan una función de activación diferenciable y que la función de error (*loss function*) también lo es. Especiﬁque explícitamente las funciones anteriores, así como sus gradientes. Escriba funciones para: 
    1. Dar valores iniciales a los pesos de la red
    2. Implementar el forward pass
    3. Implementar el backward pass
    4. Implementar la rutina principal de entrenamiento, adoptando, por simplicidad, la variante cíclica de SGD (un ejemplo a la vez, pero iterando cíclicamente sobre el conjunto de entrenamiento) con una tasa de aprendizaje y número de ciclos ﬁjos (epochs).
----------
*  Escriba una función que permita hacer predicciones mediante una red FF con 1 capa escondida (H neuronas) y O neuronas de salida, sin usar librerías, excepto eventualmente numpy. Escriba una función vectorizada que implemente el **forward pass** sobre un conjunto de *n<sub>test</sub>* ejemplos.

----------

* Demuestre que sus programas funcionan en un problema de clasiﬁcación, eligiendo funciones de error y de activación apropiadas. Para esto utilice el dataset seeds, disponible en [UCI](http://archive.ics.uci.edu/ml/) y correspondiente a la clasiﬁcación de distintos tipos de semillas (3 clases) (recuerde que usualmente es conveniente normalizar los datos antes de trabajar con el modelo).

```python
from sklearn.preprocessing import StandardScaler
import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt'
df = pd.read_csv(url, sep=r'\s+',header=None)
X_train = df.ix[:,0:6]
y_train = df.ix[:,7]
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

```
Para evaluar los resultados, construya un gráﬁco correspondiente al error de clasiﬁcación versus número de *epochs*, utilizando sólo el conjunto de entrenamiento (el objetivo de esta sección es familiarizarse con el algoritmo BP, no encontrar la mejor red). Graﬁque también la evolución de la función objetivo utilizada para el entrenamiento.

----------

* Construya, sin usar librerías, excepto eventualmente numpy para implementar operaciones básicas de álgebra lineal, una variante de su programa anterior que entrene la red utilizando *weight-decay*.

### Solución

Utilizando la [siguiente referencia](http://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/) implementaremos una red.

**A) Programa de entrenamiento**

Esta sección ha sido subdividida en cuatro etapas, las cuales involucran desde la inicialización de la red hasta la implementación de la rutina principal de entrenamiento. Para este trabajo deberemos primero crear la nueva red, donde cada neurona tiene un conjunto de pesos, durante el entrenamiento tendremos que almacenar propiedades adicionales es por este motivo que definiremos a una **neurona** como un *diccionario* y guardaremos los pesos con el nombre *weights*. Por otra parte, una red se organiza en *capas*, donde la capa de entrada (*input layer*) es una fila del *dataset de entrenamiento*. La verdadera primera capa es la capa escondida (*hidden layer*), la cual es seguida por la capa de salida (*output layer*) que contiene una neurona por cada valor de una clase. Como podemos inferir, organizaremos una capa como un arreglo (*lista*) de neuronas (*diccionarios*) y una red como un arreglo (*lista*) de capas.

```python
neuron = dict()
neuron['weights'] = [value1, value2, value3, ..., valueN]

layer = list()
...
layer = [neuron1, neuron2, neuron3, ..., neuronM]

network = list()
...
network = [layer1, layer2, layer3, ..., layerZ]
```

Para dar valores iniciales a la red de pesos utilizaremos números pequeños aleatoreos (**Libreria random**), los cuales estarán en el rango de 0 a 1.

In [7]:
#--------------------------------------------
#            initialize_network
#--------------------------------------------
#   FUNCTION_IN_PARAMETERS_DEFINITION
#   n_inputs:  integer number of inputs
#   n_hidden:  integer number of neurons to 
#              have in the hidden layer
#   n_outputs: integer number of outputs
#
#   FUNCTION_OUT_PARAMETERS_DEFINITION
#   out:    list of list of dictionaries that
#           means an array of layers of neurons
#   FUNCTION_CODING
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = list()
    for i in range(n_hidden):
        hidden_layer.append({'weights': [rd.random() for j in range(n_inputs + 1)]})
    network.append(hidden_layer)
    
    output_layer = list()
    for i in range(n_outputs):
        output_layer.append({'weights': [rd.random() for j in range(n_hidden + 1)]})
    network.append(output_layer)
    
    return network
#   FUNCTION_EXPLANATION
#   Creates a new neural network ready for 
#   training. It accepts three parameters,
#   the number of inputs, the number of neurons
#   to have in the hidden layer and the number 
#   of outputs.
#--------------------------------------------

Podemos notar que la capa escondida contiene *n<sub>hidden</sub>* neuronas y cada neurona tiene *n<sub>inputs</sub> + 1* pesos, una por cada columna en el *dataset* y uno adicional para el **bias** (*sesgo*). También se puede notar que la capa de salida que se conecta a la capa escondida presenta *n<sub>outputs</sub>* neuronas y cada una con *n<sub>hidden</sub> + 1* pesos. Esto permite que cada neurona en la capa de salida esté conectada (*tenga un peso*) con una neurona en la capa escondida.

#### Testing de la función


In [11]:
# Mantener la aleatoriedad controlada
rd.seed(1)

# Testing
network = initialize_network(2, 1, 2)
for layer in network:
    print layer

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


Se puede notar que la capa escondida tiene una neurona con 2 pesos además del sesgo y la capa de salida tiene 2 neuronas, cada una con 1 peso además del sesgo. Ahora que ya sabemos como crear e inicializar una red, veamos como calcular la salida.

----------------
#### Forward Propagate

Se puede calcular la salida de una red neuronal mediante la propagación de una señal de entrada a través de las capas hasta que la capa de salida retorne el valor.

Para este proceso tenemos tres partes:
1. Activación de la neurona
2. Transferencia de la neurona
3. Propagación hacia adelante

----------------

El primer paso es calcular la activación de una neurona dadas una entrada. Para este caso, podremos decir que la entrada es una fila del *DataSet* de entrenamiento al igual que en el caso de la capa escondida. La función para calcular es muy cercana a una regresión lineal, donde se calcula la suma de los pesos para cada entrada. El sesgo puede considerarse como aparte, o como que es multiplicado siempre por 1.

\begin{equation*}
activation = ( \sum_{k=1}^n weight_i * input_i ) + bias
\end{equation*}

Gracias a nuestra definición previa podremos siempre definir al *bias* como el último valor del arreglo de pesos, entonces una implementación de esta función es

In [12]:
#--------------------------------------------
#            activate_neuron
#--------------------------------------------
#   FUNCTION_IN_PARAMETERS_DEFINITION
#   weights:  list of floats
#   inputs:   list of floats
#
#   FUNCTION_OUT_PARAMETERS_DEFINITION
#   out:    integer value that represent the 
#           activation equation
#   FUNCTION_CODING
def activate_neuron(weights, inputs):
    activation = weights[-1]
    quantum = len(weights) - 1
    for i in range(quantum):
        activation += weights[i] * inputs[i]
        
    return activation
#   FUNCTION_EXPLANATION
#   Calculate the neuron activation for an input
#--------------------------------------------

----------------
Ahora que tenemos nuestra función de activación, veamos como utilizarla. Para poder lograr obtener el resultado deberemos utilizar la función de activación y transferir este valor a través de las capas.

Existen diferentes funciones de transferencia pero tradicionalmente se utiliza la [función Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function). Otras opciones pueden ser:

- [Tangente hiperbólica](https://en.wikipedia.org/wiki/Hyperbolic_function)
- [Rectificante](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

La activación de sigmoid, también conocida como función logística, pede tomar cualquier valor y producir un número entre 0 y 1 dentro de su curva, la cual tiene una forma parecida a una S aplastada.
<img src="https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg" width="400" height="800" />

Una particularidad de esta función es que se le puede calcular facilmente la derivada, lo cual facilitará implementaciones posteriores. Entonces, definimos la función sigmoid como

\begin{equation*}
S(t) = \frac{1}{1 + e^{-t}}
\end{equation*}

Lo cual implementaremos de la siguiente forma

In [18]:
#--------------------------------------------
#            transfer
#--------------------------------------------
#   FUNCTION_IN_PARAMETERS_DEFINITION
#   value: float number
#
#   FUNCTION_OUT_PARAMETERS_DEFINITION
#   out:    float number of transferation
#   FUNCTION_CODING
def transfer(value):
    transfer_value = 1.0 / (1 + np.exp(-value))
    
    return transfer_value
#   FUNCTION_EXPLANATION
#   Calculate the sigmoid function value of input
#--------------------------------------------

----------------
Ya habiendo definido nuestra base, la propagación es una aplicación directa. Trabajando sobre cada capa de la red se calcula la salida de cada neurona, la cual sera utilizada como input para la siguiente capa.

Debido a la estructura que elegimos para la neurona, podremos guardar el valor de salida como una llave del diccionario, además dentro de la iteración de capas deberemos almacenar todos estos valores para que al cambio de capa sean utilizados como los nuevos valores de entrada.

In [19]:
#--------------------------------------------
#            forward_propagate
#--------------------------------------------
#   FUNCTION_IN_PARAMETERS_DEFINITION
#   network: list of lists
#   row:     list of float values
#
#   FUNCTION_OUT_PARAMETERS_DEFINITION
#   out:    list of float values
#   FUNCTION_CODING
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = list()
        for neuron in layer:
            activation_value = activate_neuron(neuron['weights'], inputs)
            neuron['output'] = transfer(activation_value)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
        
    return inputs
#   FUNCTION_EXPLANATION
#   Propagates the values from a row of values
#   till the output's layer and returns the last
#   list of values
#--------------------------------------------

-----------------
Finalmente debemos realizar un testing del funcionamiento. Utilizando los valores previos de red y generando una lista a propagar de valores 1 y 0 obtendremos una salida de dos valores dado que la red que habiamos configurado contenia dos neuronal en su capa final.

In [20]:
row = [1, 0, None]
output = forward_propagate(network, row)

print output

[0.66299701298528868, 0.72531607252797481]


#### Backward propagate

Esta sección de trabajo es la que permite determinar errores comparando el valor esperado con el valor propagado de la red. Este error es revisado a través de la red desde la capa final hasta la capa escondida asignando responsabilidad sobre el error y actualizando los pesos a medida que avanza.

La propagación del error tiene una base científica en el *Cálculo*, sin embargo como simplificación nos mantendremos al margen de su funcionamiento y nos concentraremos en su uso.

Al igual que la sección anterior, podemos dividir el trabajo en dos partes

1. Derivada de transferencia
2. Error de propagación hacia atras
-----------------

Lo primero es calcular la derivada de transferencia que para este informe estamos utilizando la función **Sigmoid**, cuya derivada se expresa de la siguiente forma

\begin{equation*}
S'(t) = S(t)(1 - S(t))
\end{equation*}

Esto nos permite implementarla de una manera muy sencilla

In [21]:
#--------------------------------------------
#            transfer_derivative
#--------------------------------------------
#   FUNCTION_IN_PARAMETERS_DEFINITION
#   S:   float number representing S(t)
#
#   FUNCTION_OUT_PARAMETERS_DEFINITION
#   out:    float number 
#   FUNCTION_CODING
def transfer_derivative(S):
    derivative = S * (1.0 - S)
    
    return derivative
#   FUNCTION_EXPLANATION
#   Calculate the transfer derivative 
#--------------------------------------------

---------------
Ahora que tenemos la función implementada, debemos calcular el error para cada salida de la neurona, esto nos dará la señal de error que propagaremos hacia atrás a través de la red.

Para obtener el error que mencionamos recién es necesario realizar una comparación entre el valor esperado y el valor resultante, a lo cual además multiplicaremos la derivada de la transferencia, esto se expresa matemáticamente como

\begin{equation*}
Error = (E(x) - O(x)) * \frac{\partial Sigmoid(x)}{\partial x}
\end{equation*}

Este cálculo de error es usado para neuronas en la capa de salida, donde el valor esperado es el valor de la clase. La señal para una neurona en la capa escondida es calculada como el error de los pesos de cada neurona en la capa de salida.

Ahora bien, la señal del error propagado se acumula y usa para determinar el error de la neurona en la capa escondida de la siguiente forma

\begin{equation*}
Error = (weight_i * error_j) *
\end{equation*}